In [2]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime
import statsmodels.tsa.api as smt

import os
print(os.listdir("./"))

['sample_submission.csv', '.ipynb_checkpoints', 'shops.csv', 'test.csv', 'items.csv', 'Untitled.ipynb', 'item_categories.csv', 'sales_train.csv', '01_EDA.ipynb', 'previous-value-benchmark-simple-eda.ipynb', 'Untitled1.ipynb']


# Previous Value Benchmark

In this first approach, we must create a prediction for November 2015 using the exact historical sales data of the previous month, October 2015. No models, no training set, simply we assume that November 2015 will have the same behaviouir than October 2015. 

In [20]:
# Import data from csv files
test_data = pd.read_csv('test.csv')
submission_data = pd.read_csv('sample_submission.csv')
sales_data = pd.read_csv('sales_train.csv')

In [21]:
print(len(test_data))
test_data.head()

214200


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [22]:
print(len(submission_data))
submission_data.head()

214200


,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


As expected both df have the same amount of rows. For every ID in **test_data** we must predict **item_cnt_month**, the prediction must be imputed in **submission_data**.

Now we need to generate the predictions from historical data (Octobre 2015) 

In [23]:
sales_data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [32]:
# date_block_num: 0 -> January 2013 1-> February 2013 ...  33-> October 2015
oct_sales = sales_data[sales_data['date_block_num'] == 33]
oct_sales = oct_sales.groupby(['item_id','shop_id']).sum().reset_index()
print(len(oct_sales))
oct_sales.head()

31531


,item_id,shop_id,date_block_num,item_price,item_cnt_day
0,30,31,33,129.0,1.0
1,31,2,33,399.0,1.0
2,31,6,66,598.0,2.0
3,31,18,33,399.0,1.0
4,31,21,33,399.0,1.0


In October 2015, there isn't predictions for all Id is test_data (31531 rows v.s. 214200 rows). Now, we are going to join the two df an generate the submission file

In [69]:
test_pred = pd.merge(test_data, oct_sales, how='left', on=['item_id', 'shop_id'])
test_pred.head()

,ID,shop_id,item_id,date_block_num,item_price,item_cnt_day
0,0,5,5037,NaN,NaN,NaN
1,1,5,5320,NaN,NaN,NaN
2,2,5,5233,33.0,1199.0,1.0
3,3,5,5232,NaN,NaN,NaN
4,4,5,5268,NaN,NaN,NaN


Here we must fix some stuff in order to generate the get the test set. Replace NaN for 0, drop **date_block_num** and **item_price**, change **item_cnt_day** for ***item_cnt_month***

In [70]:
test_pred = test_pred.fillna(0)
test_pred = test_pred.drop(['date_block_num','item_price'], axis=1)
test_pred = test_pred.rename(columns={"item_cnt_day": "item_cnt_month"})

In [71]:
test_pred.describe()

,ID,shop_id,item_id,item_cnt_month
count,214200.000000,214200.000000,214200.000000,214200.000000
mean,107099.500000,31.642857,11019.398627,0.293413
std,61834.358168,17.561933,6252.644590,5.550976
min,0.000000,2.000000,30.000000,-1.000000
25%,53549.750000,16.000000,5381.500000,0.000000
50%,107099.500000,34.500000,11203.000000,0.000000
75%,160649.250000,47.000000,16071.500000,0.000000
max,214199.000000,59.000000,22167.000000,2253.000000


In [75]:
subm = test_pred.drop(['shop_id','item_id'], axis=1)
subm.to_csv('submission.csv', index = False) 

**Score:** 8.53027 
**LB position:** 6656 


In the tip giving by the organizers, they recommend to use the **clip** function. This function trim outputs to a specify threshold

In [76]:
subm['item_cnt_month'] = subm['item_cnt_month'].clip(0,20)
subm.describe()

,ID,item_cnt_month
count,214200.000000,214200.000000
mean,107099.500000,0.255649
std,61834.358168,1.089856
min,0.000000,0.000000
25%,53549.750000,0.000000
50%,107099.500000,0.000000
75%,160649.250000,0.000000
max,214199.000000,20.000000


In [77]:
subm.to_csv('submission.csv', index = False) 

**Score:** 1.16777 
**LB position:** 4300
